In [12]:
# Use python 3.11
import cv2
import mediapipe as mp      # Use version 0.10.14
import math

# Hands mediapipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.6,
    min_tracking_confidence=0.6,
)
mp_draw = mp.solutions.drawing_utils

# Chooses the camera in order saved on computer. if 1 doesnt work, use 0
cap = cv2.VideoCapture(1)

# Circle stats/variables
shape_x = 350
shape_y = 50
velocity_x = 3
velocity_y = 0
air_resistance = 0.1
gravity = 4
radius = 60
last_x_positon = 0
last_y_positon = 0


holding = False

try:
    while True:
        # Loading camera data
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        if not ret:
            break

        # Useful variables
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        result = hands.process(img_rgb)
        frame = cv2.flip(img, 1)
        h, w, _ = img.shape

        # Ground collision
        if shape_y + radius >= h:
            shape_y = h - radius
            velocity_y = -velocity_y * 0.7  # Bounce factor
        if shape_x + radius >= w:
            shape_x = w - radius
            velocity_x = -velocity_x * 0.7  # Bounce factor
        if shape_x - radius <= 0:
            shape_x = 0 + radius
            velocity_x = -velocity_x * 0.7  # Bounce factor

        # Draw shape
        if not holding:
            # Gravity
            velocity_y += gravity
            shape_x += velocity_x
            shape_y += velocity_y

            if velocity_x > 0:
                velocity_x -= air_resistance
            else:
                velocity_x += air_resistance

            # Draw
            cv2.circle(img, (int(shape_x), int(shape_y)), radius, (0, 0, 0), -1)
        else:
            last_x_positon = shape_x
            last_y_positon = shape_y

            # Change variables
            shape_x = int(thumb.x * w)
            shape_y = int(thumb.y * h)
            velocity_x = shape_x - last_x_positon
            velocity_y = shape_y - last_y_positon

            # Draw
            cv2.circle(img, (int(shape_x), int(shape_y)), radius, (0, 0, 0), -1)

        # Checks if hand is on screen
        if result.multi_hand_landmarks:
            # Runs through each point on hand (find points online)
            for handLms in result.multi_hand_landmarks:
                # Draws points
                mp_draw.draw_landmarks(img, handLms, mp_hands.HAND_CONNECTIONS)

                # assigning which hand points
                thumb = handLms.landmark[4]
                pointer = handLms.landmark[8]

                # Calculate distance in normalized coordinates
                distance_thumb_to_pointer = ((pointer.x - thumb.x)**2 + (pointer.y - thumb.y)**2)**0.5

                distance_thumb_to_circle = math.hypot(int(thumb.x*w) - shape_x, int(thumb.y*h)-shape_y)

                # Displaying text
                cv2.putText(img, f"distance_thumb_to_pointer: {distance_thumb_to_pointer}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

                cv2.putText(img, f"distance_thumb_to_circle: {distance_thumb_to_circle}", (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

                # Checks if thumb and pointer are touching, and if thumb is touching circle
                if distance_thumb_to_pointer <= 0.06 and distance_thumb_to_circle < radius:
                    holding = True
                else:
                    holding = False

        # Shows the image
        cv2.imshow("Hand Tracker", img)

        # If 'q' is pressed, end while loop
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    # Close everything
    cap.release()
    hands.close()
    cv2.destroyWindow("Hand Tracker")
    cv2.waitKey(1)

I0000 00:00:1771703754.127884 20050563 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4
W0000 00:00:1771703754.135547 20058472 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1771703754.140274 20058472 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
